In [0]:
!pip install mlflow --quiet
dbutils.library.restartPython()

In [0]:
import mlflow
email = 's.sillerom@gmail.com'
mlflow.set_experiment(f"/Users/{email}/2-entrena-tu-primer-modelo")

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [0]:
# Vamos a cargar un dataset de juguete para mostrar la creación y entrenamiento basado en experimentos
dataset = load_diabetes()

In [0]:
# Vamos a hacer un split rápido del dataset
X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
# La forma recomendada de empezar a usar el seguimiento de MLflow con Python es usar la API autolog() de MLflow.
# Con las capacidades de autoregistro de MLflow, una sola línea de código registra automáticamente el modelo resultante, los parámetros usados para crear el modelo y una puntuación del modelo.
mlflow.sklearn.autolog()

# Con autolog() activado, todos los parámetros del modelo, una puntuación del modelo y el modelo ajustado se registran automáticamente.
# También es posible pasar un nombre de ejecución como mlflow.start_run(run_name="Mi increíble ejecución"), de lo contrario, MLflow elegirá uno por nosotros.
with mlflow.start_run():
  # Establecer los parámetros del modelo.
  n_estimators = 100
  max_depth = 6
  max_features = 3

  # Crear y entrenar el modelo.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)

  # Usar el modelo para hacer predicciones en el conjunto de datos de prueba.
  predictions = rf.predict(X_test)

  # Registrar los parámetros del modelo utilizados en esta ejecución.
  mlflow.log_param("num_trees", n_estimators)
  mlflow.log_param("maxdepth", max_depth)
  mlflow.log_param("max_feat", max_features)

  # Definir una métrica para evaluar el modelo.
  mse = mean_squared_error(y_test, predictions)

  # Registrar el valor de la métrica de esta ejecución.
  mlflow.log_metric("mse", mse)

  # Convertir los residuos en un dataframe de pandas para aprovechar las capacidades gráficas
  df = pd.DataFrame(data = predictions - y_test)
  # Crear un gráfico de residuos
  plt.plot(df)
  plt.xlabel("Observación")
  plt.ylabel("Residual")
  plt.title("Residuos")

  # Guardar la figura del gráfico
  fig = plt.gcf()
  mlflow.log_figure(fig, "residuals_plot.png")

mlflow.end_run()